In [61]:
import numpy as np

Bijective string transformation
===============================

Take a (binary) word $w\in\{0,1\}$, with $w[i]$ meaning the $i$th letter of $w$, starting from $1$. The function **`transform`** implements a bijective string transform $f:\{0,1\}^*\rightarrow \{0,1\}^*$, computed as follows:
* Let $v_1=w$
* Compute $v_{i+1}$ from $v_i$ by taking the `XOR` of pairs of consecutive letters of $v_i$, that is, $|v_{i+1}| = |v_i|-1$, such that $v_{i+1}[j]=v_i[j] \oplus v_i[j+1]$, for all $j\in\{1,|v_i|-1\}$. 
* The last such word, $v_{|w|}$, will have one letter
* The value of $f(w)$ is given by concatenating the first letters of all $v_i$, that is,
$$f(w) = v_1[1]v_2[1]\cdots v_{|w|}[1].$$

$f$ has the following properties, which hold for all $w\in \{0,1\}^*$:
1. **Length preserving:** $|w| = |f(w)|$.
2. **Idempotent:** $w=f(f(w))$, in other form, $w = ff.w$ or $w=f^2.w$.
3. **Commutes(?) with reversal:** $f(f(w)^R) = (f(w^R))^R$, in other form, $fRf.w = RfR.w$.
4. **From 3. we get:** $fRfRfR.w = w$, in other form, $(fR)^3 = 1$.

In [62]:
def transform(word, row=0, full=False):
    size = len(word)
    result = np.zeros((size,size), dtype = np.int)
    if isinstance(word, str):
        start = np.fromiter(word, dtype = np.int)
    else:
        start = word
    result[0] = start
    for i in np.arange(1, size):
        result[i, :size-i] = (result[i-1, : size-i] + result[i-1, 1: size-i+1])%2
    if full:
        return result
    elif row > 0 and row < size:
        return result[row]
    else:
        return result[:,0]

## A printing method for the steps of computation of the transform. Row $i$ corresponds to $v_i$ ##

In [63]:
def triprint(matrix):
    size = matrix.shape[0]
    for i in np.arange(size):
        print(' '*i+' '.join(map(str, matrix[i,:size-i])))

## We can define order $k$ complements of a binary string, based on the transform. There are at least two straightforward choices, which both agree with order 1 complement being the usual notion of negating individual bits. ##

One choice is to negate the $k$-th bit of the transform and apply the transform to the result. The other is to negate all bits up to position $k$ in the transform, and appy the transform again. In symbols, if $C_k(v)=v[1]\cdots v[k-1]\cdot \overline{v[k]}\cdot v[k+1]\cdots v[|v|]$, then the first type of complementation of order $k$, amounts to $X_k(w) = fC_kf.w$, whereas the second type of complementation of order $k$ is defined by $Y_k(w) = fC_kC_{k-1}\cdots C_1f.w$.

In [64]:
def complement1(word, order):
    tr = transform(word)
    tr[order] = 1 - tr[order]
    return transform(tr)

In [65]:
complement1('10110',0)

array([0, 1, 0, 0, 1])

In [66]:
def complement2(word, order):
    tr = transform(word)
    tr[:order+1] = 1 - tr[:order+1]
    return transform(tr)

In [67]:
complement2('10110',0)

array([0, 1, 0, 0, 1])

In [68]:
word = '01010101'
for i in range(len(word)):
    print(complement1(word, i), '               ***             ', complement2(word, i))

[1 0 1 0 1 0 1 0]                ***              [1 0 1 0 1 0 1 0]
[0 0 0 0 0 0 0 0]                ***              [1 1 1 1 1 1 1 1]
[0 1 1 0 0 1 1 0]                ***              [1 1 0 0 1 1 0 0]
[0 1 0 0 0 1 0 0]                ***              [1 1 0 1 1 1 0 1]
[0 1 0 1 1 0 1 0]                ***              [1 1 0 1 0 0 1 0]
[0 1 0 1 0 0 0 0]                ***              [1 1 0 1 0 1 1 1]
[0 1 0 1 0 1 1 0]                ***              [1 1 0 1 0 1 0 0]
[0 1 0 1 0 1 0 0]                ***              [1 1 0 1 0 1 0 1]


In [69]:
triprint(transform('101001000101001000',full=True))

1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0
 1 1 1 0 1 1 0 0 1 1 1 1 0 1 1 0 0
  0 0 1 1 0 1 0 1 0 0 0 1 1 0 1 0
   0 1 0 1 1 1 1 1 0 0 1 0 1 1 1
    1 1 1 0 0 0 0 1 0 1 1 1 0 0
     0 0 1 0 0 0 1 1 1 0 0 1 0
      0 1 1 0 0 1 0 0 1 0 1 1
       1 0 1 0 1 1 0 1 1 1 0
        1 1 1 1 0 1 1 0 0 1
         0 0 0 1 1 0 1 0 1
          0 0 1 0 1 1 1 1
           0 1 1 1 0 0 0
            1 0 0 1 0 0
             1 0 1 1 0
              1 1 0 1
               0 1 1
                1 0
                 1


In [70]:
triprint(transform('1011010000101101',full=True))

1 0 1 1 0 1 0 0 0 0 1 0 1 1 0 1
 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1
  0 1 1 0 0 1 0 0 1 0 0 1 1 0
   1 0 1 0 1 1 0 1 1 0 1 0 1
    1 1 1 1 0 1 1 0 1 1 1 1
     0 0 0 1 1 0 1 1 0 0 0
      0 0 1 0 1 1 0 1 0 0
       0 1 1 1 0 1 1 1 0
        1 0 0 1 1 0 0 1
         1 0 1 0 1 0 1
          1 1 1 1 1 1
           0 0 0 0 0
            0 0 0 0
             0 0 0
              0 0
               0


In [71]:
triprint(transform('1011011101001011101101',full=True))

1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 0 1
 1 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1
  0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0
   1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 1
    1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1
     0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0
      1 1 0 0 1 1 0 0 0 0 1 1 0 0 1 1
       0 1 0 1 0 1 0 0 0 1 0 1 0 1 0
        1 1 1 1 1 1 0 0 1 1 1 1 1 1
         0 0 0 0 0 1 0 1 0 0 0 0 0
          0 0 0 0 1 1 1 1 0 0 0 0
           0 0 0 1 0 0 0 1 0 0 0
            0 0 1 1 0 0 1 1 0 0
             0 1 0 1 0 1 0 1 0
              1 1 1 1 1 1 1 1
               0 0 0 0 0 0 0
                0 0 0 0 0 0
                 0 0 0 0 0
                  0 0 0 0
                   0 0 0
                    0 0
                     0
